### Distribution of topics
+ creating data frame from TM composition file

In [ ]:
import re
import pickle
import pandas as pd

In [ ]:
import numpy as np
import pandas as pd

import matplotlib as mpl
import matplotlib.pyplot as plt

import seaborn as sns
import seaborn.objects as so

In [ ]:
year_sp = []
factionID = []
speech_id = []
count = 0

with open(f'proc_data/topics30_composition.txt', 'r', encoding='UTF-8') as f:
    
    for line in f:
        
        data = line.strip().split("\t")
        topics = data[2:]
        sp_id = data[1] 
        
        #find position of sp_id
        sp_id_index = [i for i, e in enumerate(sp_id) if e == '_']
        
        #slincing for sp_id
        sp_ID = sp_id[sp_id_index[0]+1:sp_id_index[1]]
        speech_id.append(sp_ID)
        
        #find position of year
        year_index = [i for i, e in enumerate(sp_id) if e == '/']
        
        #slicing for year
        year = int(sp_id[year_index[7]+1:year_index[8]])
        year_sp.append(year)
        
        #find position of faction id
        faction_index = [i for i, e in enumerate(sp_id) if e == '_']
        
        #slicing for faction id
        faction_id = int(sp_id[faction_index[2]+1:-4])
        factionID.append(faction_id)
        
        topics = [float(t) for t in topics]

+ putting together the data frame 

In [ ]:
columns = ["index", "filename"] 

sum_topics = []

for i in range(30):
    columns.append(f'topic{i}')
    
df = pd.read_csv(f'proc_data/topics30_composition.txt', sep="\t", names=columns)

# adding speechID, year and faction
#df.insert(loc, column_name, value)
df.insert(2, "speechID", speech_id)
df.insert(3, "year", year_sp)
df.insert(4, "faction", factionID)


for i in range(30):
    sum_topics.append(sum(df[f'topic{i}']))

In [ ]:
df = df.drop(columns=["filename"])

In [ ]:
df

### Distribution for all factions
+ grouping data by year
+ building new index over year (73 years)
+ normalizing topic weights
+ create data frame with normalized topic weights per year (norm_mean_new)

In [ ]:
col_names = []
for col_name in df.columns:
    if 'topic' in col_name:
        col_names.append(col_name)
norm_weight_year = df.groupby('year')[col_names].sum().reset_index()
norm_weight_year[col_names] /= norm_weight_year[col_names].sum()
norm_weight_year.columns = ['year'] + [f'{topic}_weighted' for topic in col_names]

In [ ]:
norm_weight_year

+ calculate rolling mean
+ clear data frame from NA and correct axis years

In [ ]:
norm_mean = norm_weight_year.rolling(8).mean()

In [ ]:
norm_mean_new = norm_mean.dropna(how='all')

In [ ]:
norm_mean_new['year']+=4

In [ ]:
norm_mean_new

### Distribution SPD
+ filtering faction 23 (SPD)
+ create data frame from df for SPD (norm_mean_SPD_new)

In [ ]:
df['year']+=4

In [ ]:
df_SPD = df[(df['faction'] == 23)]

In [ ]:
df_SPD

In [ ]:
col_names = []
for col_name in df.columns:
    if 'topic' in col_name:
        col_names.append(col_name)
norm_weight_SPD = df_SPD.groupby('year')[col_names].sum().reset_index()
norm_weight_SPD[col_names] /= norm_weight_SPD[col_names].sum()
norm_weight_SPD.columns = ['year'] + [f'{topic}_weighted' for topic in col_names]

In [ ]:
norm_weight_SPD

In [ ]:
norm_mean_SPD_new = norm_weight_SPD.rolling(8).mean()

In [ ]:
norm_mean_SPD_new = norm_mean_SPD_new.dropna(how='all')

In [ ]:
norm_mean_SPD_new = norm_mean_SPD_new.astype({'year': 'int32'})

In [ ]:
norm_mean_SPD_new['year']+=4

In [ ]:
norm_mean_SPD_new

### Distribution CDU/ CSU
+ filtering faction 4 (CDU + CSU)
+ create data frame from df for CDU + CSU (norm_mean_CDU_new)

In [ ]:
df_CDU = df[(df['faction'] == 4)]

In [ ]:
col_names = []
for col_name in df.columns:
    if 'topic' in col_name:
        col_names.append(col_name)
norm_weight_CDU = df_CDU.groupby('year')[col_names].sum().reset_index()
norm_weight_CDU[col_names] /= norm_weight_CDU[col_names].sum()
norm_weight_CDU.columns = ['year'] + [f'{topic}_weighted' for topic in col_names]

In [ ]:
norm_mean_CDU_new = norm_weight_CDU.rolling(8).mean()

In [ ]:
norm_mean_CDU_new = norm_mean_CDU_new.dropna(how='all')

In [ ]:
norm_mean_CDU_new = norm_mean_CDU_new.astype({'year': 'int32'})

In [ ]:
norm_mean_CDU_new

### Example Plotting
+ similar plotting code was used for all graphs

In [ ]:
f, ax = plt.subplots(1, 1)
x_col='year'
y_col = 'topic weight'

ax.plot(norm_mean_new.year, norm_mean_new["topic5_weighted"], color="blue", label="Immigration (5)", linestyle="-")
ax.plot(norm_mean_new.year, norm_mean_new["topic21_weighted"], color="orchid", label="German unification (21)", linestyle="-")
ax.plot(norm_mean_new.year, norm_mean_new["topic10_weighted"], color="seagreen", label="Legislation (10)", linestyle="-")
ax.plot(norm_mean_new.year, norm_mean_new["topic3_weighted"], color="orange", label="Development (3)", linestyle="-")
ax.plot(norm_mean_new.year, norm_mean_new["topic27_weighted"], color="grey", label="Law&Order (27)", linestyle="-")

ax.set_ylabel('Topic proportion')
ax.set_xlabel('Year')
ax.legend()

plt.show()